In [ ]:
import gradio as gr
from neo4j import GraphDatabase
from langchain_groq import ChatGroq
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph as LangChainNeo4jGraph


In [ ]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

In [43]:
# MAIN
def create_graph_rag_chain():
    # creating the main QA chaint that connect to Neo4j and uses and LLM
    
    if not GROQ_API_KEY or not NEO4J_PASSWORD:
        raise ValueError("API key or password not found")
    
    # Initialize connection to Neo4j
    graph = LangChainNeo4jGraph(
        url = NEO4J_URI,
        username = NEO4J_USERNAME,
        password = NEO4J_PASSWORD,
        database = NEO4J_DATABASE
    )

    # refreshing the schema to make sure that LLM has lastest graph structure
    graph.refresh_schema()

    # Initialize the LLM
    llm = ChatGroq(temperature=0, model = "meta-llama/llama-4-maverick-17b-128e-instruct" )

    # Creating GraphCypherQAChain
    # It will take a question, generate Cypher query, execute it
    # andthen use the result to answer the orignal question
    chain = GraphCypherQAChain.from_llm(
        cypher_llm = llm,
        qa_llm = llm,
        graph = graph,
        verbose = True, # true to see the generated cypher queries
        return_intermediate_steps = True, # helps in debugging
        allow_dangerous_requests=True # Acknowledges the security risk of LLM-generated queries
    )
    return chain

def query_graph(question, history):
    # function to be called by gradio interface. It takes a user questoin
    # queries the knowledge graph and returns the answer

    try:
        #geenrate a response using the GraphRAG Chain
        result = rag_chain.inovke({"query": question})

        #extract the final answer and the intermediate cypher query for dispaly
        answer = result["result"]
        intermediate_steps = result["intermediate_steps"]
        cypher_query = ""
        if intermediate_steps and len(intermediate_steps) > 0:
            cypher_query = intermediate_steps[0].get("query", "could not generate cypher query")

        
        # format the output for the UI
        formatted_output = (
            f"**Answer:**\n{answer}\n\n"
            f"--- \n"
            f"**Generated Cypher Query:**\n```cypher\n{cypher_query}\n```"

        )
        return formatted_output
    
    except Exception as e:
        print(f"Error during query: {e}")
        return f"An error occured: {e}"
    

# Gradio Interface
if __name__ == "__main__":
    print("Initializing GraphRAG chain-")
    try:
        rag_chain = create_graph_rag_chain()
        print("Chain Initialized")

        # Setup  the Gradio Interface
        with gr.blocks(theme=gr.themes.Soft(), title="Cancer Monograph GraphRAG") as demo:
            gr.Markdown(
                 """
                 Ask a question about the ICMR Cancer Monograph, and the system will query the knowledge graph to find the answer
                 **Example:**
                 - What are the leading sites of cancer in Mumbai?
                 - Which risk factors are associated with Breast Cancer?
                """
            )
            gr.chat_interface(
                fn=query_graph,
                title="ask the monograph",
                chatbot=gr.Chatbot(height=500),
                textbox=gr.Textbox(placeholder="e.g., what is the role of tobacco in oral cancer?", container=False, Scale=7),
                examples=[
                    "What are the leading sites of cancer in Mizoram?",
                    "Which studies did the ICMR conduct on tobacco?",
                    "What is the link between the GSTM1 gene and oral cancer?"
                ]
            )

        
        print("\nLaunching Gradio App, open the URL")
        demo.launch(share=True) # share = true, creates a public link

    except Exception as e:
        print(f" failed to initialize the application : {e}")

Initializing GraphRAG chain-
Chain Initialized
 failed to initialize the application : 'module' object is not callable
